In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

#wczytywanie danych
def load_images_from_folder(folder, label):
    features = []
    labels = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        img = cv2.imread(path)
        if img is None:
            continue
        img = cv2.resize(img, (128, 128))
        
       
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mean_color = cv2.mean(img)[:3]
        mean_hsv = cv2.mean(hsv)[:3]
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #tekstura
        texture = np.std(gray) 
        
        feat = np.hstack([mean_color, mean_hsv, texture])
        features.append(feat)
        labels.append(label)
    return features, labels

fresh_path = 'data/fresh'
rotten_path = 'data/rotten'

X1, y1 = load_images_from_folder(fresh_path, 0)  #0 = świeża
X2, y2 = load_images_from_folder(rotten_path, 1) #1 = zgniła

X = np.vstack([X1, X2])
y = np.hstack([y1, y2])

#podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

#prezentacja wyników
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         5

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9



In [2]:
import joblib

#zapis modelu do pliku
joblib.dump(model, "best_model.pkl")
print("Model zapisany do pliku best_model.pkl")

Model zapisany do pliku best_model.pkl


In [3]:
import cv2
import numpy as np
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
from ipywidgets import FileUpload, VBox, Output
from IPython.display import display

#Wczytanie wytrenowanego modelu RandomForest
#model zapisany w pikles best_model.pkl
import joblib
model = joblib.load("best_model.pkl")  #jak nie przejdzie to wygenerować nowyprint("Model Random Forest wczytany poprawnie!")

#funkcja ekstrakcji cech
def extract_features_from_cv2(img):
    img = cv2.resize(img, (128, 128))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mean_color = cv2.mean(img)[:3]
    mean_hsv = cv2.mean(hsv)[:3]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    texture = np.std(gray)
    feat = np.hstack([mean_color, mean_hsv, texture])
    return feat.reshape(1, -1)

#klasy
CLASS_NAMES = ["świeża ", "zgniła "]

#widget do uploadu
uploader = FileUpload(accept='image/*', multiple=False)
output = Output()

def on_upload_change(change):
    output.clear_output()
    if not uploader.value:
        return

    for file_info in uploader.value:
        #wczytaj obraz z uploadu
        image_pil = Image.open(BytesIO(file_info['content'])).convert("RGB")
        img_cv2 = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

        
        features = extract_features_from_cv2(img_cv2)

        #predykcja modelem Random Forest
        pred = model.predict(features)[0]
        prob = model.predict_proba(features)[0][pred]

        #wizualizacja i wynik
        with output:
            plt.imshow(image_pil)
            plt.axis('off')
            plt.title("Wgrane zdjęcie")
            plt.show()

            print(f"Model przewiduje: {CLASS_NAMES[pred]}")
            print(f"Pewność predykcji: {prob:.2%}")

uploader.observe(on_upload_change, names='value')
display(VBox([uploader, output]))


Model Random Forest wczytany poprawnie!
